In [1]:
import os
from typing import TypedDict, List, Union
from langchain_core.messages import HumanMessage, AIMessage
from langchain.chat_models import init_chat_model
from langgraph.graph import StateGraph, START, END

In [2]:
class AgentState(TypedDict):
    messages: List[Union[HumanMessage, AIMessage]]

In [3]:
llm = init_chat_model(
    "anthropic.claude-3-haiku-20240307-v1:0",
    model_provider="bedrock_converse",
)

In [7]:
def process(state: AgentState) -> AgentState:
    """This node will solve the request you input"""
    response = llm.invoke(state["messages"])

    state["messages"].append(AIMessage(content=response.content)) 
    print(f"\nAI: {response.content}")
    print("CURRENT STATE: ", state["messages"])

    return state

In [5]:
graph = StateGraph(AgentState)
graph.add_node("process", process)
graph.add_edge(START, "process")
graph.add_edge("process", END) 
agent = graph.compile()

In [11]:
conversation_history = []

In [12]:
user_input = input("Enter: ")
print(f"Human: {user_input}")
while user_input != "exit":
    conversation_history.append(HumanMessage(content=user_input))
    result = agent.invoke({"messages": conversation_history})
    conversation_history = result["messages"]
    user_input = input("Enter: ")
    print(f"\nHuman: {user_input}")


Human: Hi

AI: Hello! How can I assist you today?
CURRENT STATE:  [HumanMessage(content='Hi', additional_kwargs={}, response_metadata={}), AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={})]

Human: My name is John

AI: It's nice to meet you, John! I'm glad we've had the chance to introduce ourselves. As an AI assistant, my role is to try my best to help you with any questions or tasks you may have. Please feel free to let me know if there is anything I can assist you with.
CURRENT STATE:  [HumanMessage(content='Hi', additional_kwargs={}, response_metadata={}), AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={}), HumanMessage(content='My name is John', additional_kwargs={}, response_metadata={}), AIMessage(content="It's nice to meet you, John! I'm glad we've had the chance to introduce ourselves. As an AI assistant, my role is to try my best to help you with any questions or tasks you may h

In [15]:
with open("logging.txt", "w") as file:
    file.write("Your Conversation Log:\n")
    
    for message in conversation_history:
        if isinstance(message, HumanMessage):
            file.write(f"You: {message.content}\n")
        elif isinstance(message, AIMessage):
            file.write(f"AI: {message.content}\n\n")
    file.write("End of Conversation")

print("Conversation saved to logging.txt")

Conversation saved to logging.txt
